In [1]:
import sys
import pathlib
import torch

from torch.utils.data import Dataset

In [2]:
sys.path.insert(0, str(pathlib.Path().resolve().parent))

In [3]:
from domain_adaptation.cyclegan.dataloaders.base_dataloader import UnpairedDataset, create_dataloader
from datasets_adapters.fetal_planes_db.fpd_dataset import FetalPlanesDBDataset
from datasets_adapters.fetal_head_circ.fhc_dataset import FetalHeadCircDataset
from quality.fid import calculate_fid_from_datasets

In [4]:
device = torch.device('cuda')

In [5]:
dataset_a = FetalPlanesDBDataset(
    root = '/home/jupyter/datasphere/project/ultrasound/datasetss/fetal_planes_db',
    transform = None,
    target_size = (224, 224),
    csv_file = 'FETAL_PLANES_DB_data.csv',
    images_dir = 'Images',
    train = None,
)

dataset_b = FetalHeadCircDataset(
    images_dir = '/home/jupyter/datasphere/project/ultrasound/datasetss/fetal_head_circumference/training_set',
    csv_file = '/home/jupyter/datasphere/project/ultrasound/datasetss/fetal_head_circumference/training_set_pixel_size_and_HC.csv',
    transform = None,
    target_size = (224, 224),
    load_annotations = True
)

Loaded 12400 images from /home/jupyter/datasphere/project/ultrasound/datasetss/fetal_planes_db
Loaded 999 images from /home/jupyter/datasphere/project/ultrasound/datasetss/fetal_head_circumference/training_set
Found 999 annotation images


In [6]:
initial_fid_score = calculate_fid_from_datasets(dataset_a, dataset_b, device=device, image_size=224)

Calculating FID between two datasets...
Dataset 1: 12400 samples
Dataset 2: 999 samples
Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /tmp/xdg_cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth


100%|██████████| 97.8M/97.8M [00:01<00:00, 55.0MB/s]



Extracting features from dataset 1...
Using device: cuda
Extracting features...


Processing batches: 100%|██████████| 388/388 [00:35<00:00, 10.87it/s]


Extracted 12400 features of dimension 2048

Extracting features from dataset 2...
Using device: cuda
Extracting features...


Processing batches: 100%|██████████| 32/32 [00:03<00:00,  8.12it/s]


Extracted 999 features of dimension 2048

Calculating FID score...


In [7]:
print(f'initial_fid_score: {initial_fid_score}')

initial_fid_score: 67.1516130298318


In [6]:
from domain_adaptation.cyclegan.train import CycleGANTrainer
from domain_adaptation.cyclegan.model import CycleGAN
from domain_adaptation.cyclegan.dataloaders.translated_dataset import CycleGANTranslatedDataset

In [9]:
!ls ./checkpoints/cyclegan

best_model.pt
checkpoint_epoch_1.pt
checkpoint_epoch_10.pt
checkpoint_epoch_12.pt
checkpoint_epoch_13.pt
checkpoint_epoch_14.pt
checkpoint_epoch_16.pt
checkpoint_epoch_18.pt
checkpoint_epoch_19.pt
checkpoint_epoch_2.pt
checkpoint_epoch_20.pt
checkpoint_epoch_3.pt
checkpoint_epoch_4.pt
checkpoint_epoch_6.pt
checkpoint_epoch_8.pt
samples


In [7]:
# load cyclegan
model = CycleGAN(
    input_channels_a = 1,
    input_channels_b = 1,
    n_residual_blocks = 3,
)
trainer = CycleGANTrainer(
    model=model,
    device=torch.device('cpu'),
    lambda_cycle=10,
    lambda_identity=0.5,
    lr_g=2e-4,
    lr_d=2e-4
)
checkpoint = trainer.load_checkpoint('./checkpoints/cyclegan/best_model.pt')

In [18]:
dataset_a_to_b = CycleGANTranslatedDataset(
    cyclegan_model = model,
    source_dataset = dataset_a,
    device = torch.device('cpu'))

In [19]:
a_to_b_fid_score = calculate_fid_from_datasets(dataset_a_to_b, dataset_b, device=device, image_size=224)

Calculating FID between two datasets...
Dataset 1: 12400 samples
Dataset 2: 999 samples

Extracting features from dataset 1...
Using device: cuda
Extracting features...


Processing batches: 100%|██████████| 388/388 [21:08<00:00,  3.27s/it]


Extracted 12400 features of dimension 2048

Extracting features from dataset 2...
Using device: cuda
Extracting features...


Processing batches: 100%|██████████| 32/32 [00:04<00:00,  7.69it/s]

Extracted 999 features of dimension 2048

Calculating FID score...


In [20]:
print(f'a_to_b_fid_score: {a_to_b_fid_score}')

a_to_b_fid_score: 46.15082634065115


In [8]:
dataset_b_to_a = CycleGANTranslatedDataset(
    cyclegan_model = model,
    source_dataset = dataset_b,
    device = torch.device('cpu'),
    b2a = True)

In [9]:
b_to_a_fid_score = calculate_fid_from_datasets(dataset_a, dataset_b_to_a, device=device, image_size=224)

Calculating FID between two datasets...
Dataset 1: 12400 samples
Dataset 2: 999 samples
Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /tmp/xdg_cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth


100%|██████████| 97.8M/97.8M [00:02<00:00, 41.0MB/s]



Extracting features from dataset 1...
Using device: cuda
Extracting features...


Processing batches: 100%|██████████| 388/388 [00:37<00:00, 10.39it/s]


Extracted 12400 features of dimension 2048

Extracting features from dataset 2...
Using device: cuda
Extracting features...


Processing batches: 100%|██████████| 32/32 [01:55<00:00,  3.60s/it]

Extracted 999 features of dimension 2048

Calculating FID score...


In [10]:
print(f'b_to_a_fid_score: {b_to_a_fid_score}')

b_to_a_fid_score: 81.69116442403748


In [20]:
from datasets_adapters.dogs.dogs_dataset import DogsDataset

In [21]:
doggies = DogsDataset(
    root='/home/jupyter/datasphere/project/ultrasound/datasetss/dogs_dataset',
)

Loaded 494 images across 4 breeds from /home/jupyter/datasphere/project/ultrasound/datasetss/dogs_dataset


In [22]:
a_to_dog_fid = calculate_fid_from_datasets(dataset_a, doggies, device=device, image_size=224)

Calculating FID between two datasets...
Dataset 1: 12400 samples
Dataset 2: 494 samples

Extracting features from dataset 1...
Using device: cuda
Extracting features...


Processing batches: 100%|██████████| 388/388 [00:36<00:00, 10.56it/s]


Extracted 12400 features of dimension 2048

Extracting features from dataset 2...
Using device: cuda
Extracting features...


Processing batches:  62%|██████▎   | 10/16 [00:05<00:02,  2.24it/s]/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
Processing batches: 100%|██████████| 16/16 [00:10<00:00,  1.49it/s]

Extracted 494 features of dimension 2048

Calculating FID score...


In [24]:
print(f'a_to_dog_fid: {a_to_dog_fid}')

a_to_dog_fid: 241.54387621118366


In [25]:
b_to_dog_fid = calculate_fid_from_datasets(dataset_b, doggies, device=device, image_size=224)

Calculating FID between two datasets...
Dataset 1: 999 samples
Dataset 2: 494 samples

Extracting features from dataset 1...
Using device: cuda
Extracting features...


Processing batches: 100%|██████████| 32/32 [00:05<00:00,  6.38it/s]


Extracted 999 features of dimension 2048

Extracting features from dataset 2...
Using device: cuda
Extracting features...


Processing batches:  62%|██████▎   | 10/16 [00:05<00:02,  2.48it/s]/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
Processing batches: 100%|██████████| 16/16 [00:09<00:00,  1.67it/s]

Extracted 494 features of dimension 2048

Calculating FID score...


In [26]:
print(f'b_to_dog_fid: {b_to_dog_fid}')

b_to_dog_fid: 244.2196451284496
